## 4. Training

In this part of notebook we will try to train the model using feature extracted dataframe and make model evaluation

### Setup Prerequisite

In [1]:
!pip install pyspark

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import Needed Library, Initialize Spark and Load Dataframe

In [26]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

In [4]:
spark = SparkSession.builder.appName('sparkify-train').getOrCreate()

In [5]:
# load data and change is_churn column into label column

isOnColab = True # CHANGE THIS VARIABLE IF RUNNING ON DATAPROC

path = '/content/drive/MyDrive/datasets/dsnd-sparkify/ml_df.parquet' if isOnColab else 'gs://udacity-dsnd/ml_df.parquet'
df = spark.read.parquet(path)
df = df.withColumn('label', F.when(F.col("is_churn"), 1).otherwise(0))
df.show(5)

+-------+-------------------+--------+----------+-------------+------------------+----------+-----------+---------+------------------+----------+-----+------+-----+-----+-------+-------+------------+-------------+----------+------------+-----+
| userId|              up_ts|is_churn|song_count|subs_duration|         song_rate|n_playlist|thumbs_down|thumbs_up|      avg_sess_len|sess_count| ipad|iphone|linux|macos|windows|n_error|n_friend_add|n_cancel_page|n_unq_song|n_unq_artist|label|
+-------+-------------------+--------+----------+-------------+------------------+----------+-----------+---------+------------------+----------+-----+------+-----+-----+-------+-------+------------+-------------+----------+------------+-----+
|1071843|2018-11-08 13:16:59|   false|      1190|           22| 54.09090909090909|        39|          6|       51|27020.272727272728|        11|false| false|false| true|  false|      1|          32|            0|      1104|         902|    0|
|1120784|2018-10-11 14:1

In [6]:
# features columns without state one-hot encoding
feature_cols = df.columns[3:-1]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

df = assembler.transform(df)
df = df.select(["features", "label"])

In [7]:
train_df, test_df = df.randomSplit([0.9,0.1], seed=42)

In [8]:
train_df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 4470|
|    0| 9148|
+-----+-----+



In [9]:
test_df.groupby("label").count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  480|
|    0| 1037|
+-----+-----+



In [10]:
gbt = GBTClassifier()

gbt_grid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [4, 5, 6]) \
    .addGrid(gbt.maxIter, [15, 20, 25]) \
    .build()

gbt_tvs = TrainValidationSplit(estimator=gbt,
                            estimatorParamMaps=gbt_grid,
                            evaluator=BinaryClassificationEvaluator(),
                            trainRatio=0.75,
                            seed=42)

In [11]:
# train model
gbt_model = gbt_tvs.fit(train_df)

In [12]:
print(gbt_model.bestModel)

GBTClassificationModel: uid = GBTClassifier_de3b32670a8c, numTrees=25, numClasses=2, numFeatures=18


In [30]:
# predict on test dataframe
preds_df = gbt_model.transform(test_df)

In [14]:
gbt_model.bestModel.featureImportances

SparseVector(18, {0: 0.0001, 1: 0.3445, 2: 0.1394, 3: 0.0052, 4: 0.0234, 5: 0.0262, 6: 0.0928, 7: 0.1744, 8: 0.0003, 13: 0.0009, 14: 0.0033, 15: 0.1882, 16: 0.0013})

In [ ]:
def evaluate(df, label_col='label', pred_col='prediction'):
    temp_df = df.select([label_col, pred_col]).toPandas()
    
    return pd.DataFrame.from_dict({
        "accuracy" : [accuracy_score(temp_df[label_col], temp_df[pred_col])],
        "precision" : [precision_score(temp_df[label_col], temp_df[pred_col])],
        "f1" : [f1_score(temp_df[label_col], temp_df[pred_col])],
        "roc_auc" : [roc_auc_score(temp_df[label_col], temp_df[pred_col])],
    })

In [31]:
evaluate(preds_df)

,accuracy,precision,f1,roc_auc
0,0.78708,0.663202,0.663892,0.754182


In [ ]:
model.save('/content/drive/MyDrive/datasets/dsnd-sparkify/sparkify_model')